In [392]:
import pandas as pd
# ! pip install unidecode

In [393]:
bigrams = pd.read_csv("bigram_data.csv")
bigrams.head()

,custom_id,twitter_handle,bigram
0,263,RepFinkenauer,"(health,care)"
1,263,RepFinkenauer,"(care,act)"
2,263,RepFinkenauer,"(million,americans)"
3,263,RepFinkenauer,"(pre,existing)"
4,263,RepFinkenauer,"(existing,conditions)"


In [394]:
handles = pd.read_csv("Congressional Record.csv")
handles.head()

,Congress_id,Name,State,Twitter,Facebook,Position,Party
0,263,Abby Finkenauer,Iowa 1st District,@RepFinkenauer,https://www.facebook.com/RepAbbyFinkenauer,House,Democratic Party
1,516,Abigail Spanberger,Virginia 7th District,@RepSpanberger,https://www.facebook.com/RepAbigailSpanberger,House,Democratic Party
2,242,Adam Kinzinger,Illinois 16th District,@RepKinzinger,https://www.facebook.com/RepKinzinger,House,Republican Party
3,142,Adam Schiff,California 28th District,@RepAdamSchiff,https://www.facebook.com/RepAdamSchiff,House,Democratic Party
4,528,Adam Smith,Washington 9th District,@RepAdamSmith,https://www.facebook.com/RepAdamSmith,House,Democratic Party


In [395]:
scores = pd.read_csv("trump_score_latest.csv")
scores.head()

,congress,chamber,bioguide,last_name,state,district,party,votes,agree_pct,predicted_agree,net_trump_vote
0,0,house,A000055,Aderholt,AL,4.0,Republican,122,0.975410,0.956077,63.0
1,115,house,A000055,Aderholt,AL,4.0,R,95,0.968421,0.946349,63.0
2,116,house,A000055,Aderholt,AL,4.0,R,27,1.000000,0.990306,63.0
3,0,house,A000367,Amash,MI,3.0,Republican,120,0.616667,0.792576,9.4
4,115,house,A000367,Amash,MI,3.0,R,96,0.541667,0.847386,9.4


In [429]:
journ = pd.read_csv("Journalist_bigrams.csv")
journ.head()

,custom_id,twitter_handle,bigram,custom_id-2,twitter_handle-2,fullname,newssource,newssource_party,newsspource_lean
0,2001,BenWeiserNYT,"(white,house)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
1,2001,BenWeiserNYT,"(trump,administration)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
2,2001,BenWeiserNYT,"(trump,administration)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
3,2001,BenWeiserNYT,"(public,service)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
4,2001,BenWeiserNYT,"(white,house)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid


In [433]:
def separate_by_id(bigrams_df):
    df_l = []
    for cid in bigrams_df["custom_id"].unique():
        df = bigrams_df[bigrams_df["custom_id"]==cid]
        df_l.append(df)
    return df_l

In [397]:
# add Trump Score to handles

no_match = []
one_match = []
many_match = []
for index in scores.index:
    row = scores.loc[index]
    matches = handles[handles["Name"].str.contains(row["last_name"])]
    party = row["party"][0]
    chamber = row["chamber"][0].upper()
    state = row["state"][0]
    matches = matches[(matches["Party"].str.startswith(party))
       & (matches["Position"].str.startswith(chamber))
       & (matches["State"].str.startswith(state))]
    if (len(matches) == 0) and row["congress"] == 116:
        no_match.append(row)
    elif len(matches) == 1:
        handles.at[matches.index[0], "score_{}".format(row["congress"])] = row["agree_pct"]
        one_match.append((row, matches))
    elif len(matches) > 1:
        many_match.append((row, matches))
        
        
print("No Matches:\t", len(no_match))
print("One Match:\t", len(one_match))
print("Many Matches:\t", len(many_match))

No Matches:	 6
One Match:	 1485
Many Matches:	 12


In [398]:
handles.at[312, "score_116"] = no_match[0]["agree_pct"]

In [399]:
handles.at[196, "score_116"] = no_match[2]["agree_pct"]

In [400]:
handles.at[37, "score_116"] = no_match[3]["agree_pct"]

In [401]:
handles.at[116, "score_116"] = no_match[4]["agree_pct"]

In [402]:
handles.at[447, "score_116"] = no_match[5]["agree_pct"]

In [403]:
handles.at[65, "score_0"] = many_match[0][0]["agree_pct"]

In [404]:
handles.at[65, "score_115"] = many_match[1][0]["agree_pct"]

In [405]:
handles.at[65, "score_116"] = many_match[2][0]["agree_pct"]

In [406]:
handles.at[373, "score_0"] = many_match[3][0]["agree_pct"]

In [407]:
handles.at[373, "score_115"] = many_match[4][0]["agree_pct"]

In [408]:
handles.at[373, "score_116"] = many_match[5][0]["agree_pct"]

In [409]:
handles.at[74, "score_0"] = many_match[6][0]["agree_pct"]

In [410]:
handles.at[74, "score_115"] = many_match[7][0]["agree_pct"]

In [411]:
handles.at[74, "score_116"] = many_match[8][0]["agree_pct"]

In [412]:
handles.at[5, "score_0"] = many_match[9][0]["agree_pct"]

In [413]:
handles.at[5, "score_115"] = many_match[10][0]["agree_pct"]

In [414]:
handles.at[5, "score_116"] = many_match[11][0]["agree_pct"]

In [416]:
df_l = separate_by_id(bigrams)

In [420]:
def bigram_freq(df_list: [pd.DataFrame], handles: [pd.DataFrame],
                bigrams: [str]):
    dict_list = []
    for df in df_list:
        v = df["bigram"].value_counts()
        d = {"custom_id": df["custom_id"].iloc[0]}
        try:
            handle = handles[handles["Congress_id"]==d["custom_id"]]
            d["name"] = handle["Name"].item().strip()
            d["state"] = handle["State"].item().strip()
            d["position"] = handle["Position"].item().strip()[0]
            d["party"] = handle["Party"].item().strip()[0]
            d["score_0"] = handle["score_0"].item()
            d["score_115"] = handle["score_115"].item()
            d["score_116"] = handle["score_116"].item()
        except KeyError as e:
            pass
        for bigram in bigrams:
            d[bigram] = 0
            if bigram in v:
                d[bigram] = v[bigram]
        dict_list.append(d)
    return dict_list

In [440]:
journlist = separate_by_id(journ)
journlist[0].head()

,custom_id,twitter_handle,bigram,custom_id-2,twitter_handle-2,fullname,newssource,newssource_party,newsspource_lean
0,2001,BenWeiserNYT,"(white,house)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
1,2001,BenWeiserNYT,"(trump,administration)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
2,2001,BenWeiserNYT,"(trump,administration)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
3,2001,BenWeiserNYT,"(public,service)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid
4,2001,BenWeiserNYT,"(white,house)",2001,BenWeiserNYT,Benjamin Weiser,nytimes,D,d_solid


In [452]:
dict_list = []
for df in journlist:
    v = df["bigram"].value_counts()
    d = {"custom_id": df["custom_id"].iloc[0]}
    d["name"] = df["fullname"].iloc[0].strip()
    d["state"] = df["newssource"].iloc[0].strip()
    d["position"] = "J"
    d["party"] = df["newssource_party"].iloc[0].strip()
    d["score_0"] = df["newsspource_lean"].iloc[0].strip()
    d["score_115"] = None
    d["score_116"] = None
    for bigram in bigrams["bigram"].value_counts().index:
        d[bigram] = 0
        if bigram in v:
            d[bigram] = v[bigram]
    dict_list.append(d)

In [453]:
journdf = pd.DataFrame(dict_list).set_index("custom_id")
journdf.head()
journdf.to_csv("journalist_bigram_count.csv")

In [421]:
dl = bigram_freq(df_l, handles, bigrams["bigram"].value_counts().index)

In [458]:
bigram_count = pd.DataFrame(dl).set_index("custom_id")
print("\t\t\t\t\t\tBigram Frequencies")
bigram_count.head()

						Bigram Frequencies


,"(000,jobs)","(100,days)","(116th,congress)","(2019,congressional)","(21st,century)","(6,million)","(9,11)","(abortion,survivors)","(access,quality)","(act,2019)",...,"(watch,live)","(white,house)","(women,act)",name,party,position,score_0,score_115,score_116,state
custom_id,,,,,,,,,,,,,,,,,,,,,
263,0,2,0,0,1,0,0,0,0,1,...,0,2,0,Abby Finkenauer,D,H,0.000000,NaN,0.000000,Iowa 1st District
516,0,7,2,2,3,0,0,0,1,0,...,4,0,0,Abigail Spanberger,D,H,0.000000,NaN,0.000000,Virginia 7th District
242,1,0,0,1,0,0,0,0,0,0,...,1,1,0,Adam Kinzinger,R,H,0.934426,0.989583,0.730769,Illinois 16th District
142,0,0,0,0,0,0,3,0,0,0,...,0,9,0,Adam Schiff,D,H,0.178862,0.229167,0.000000,California 28th District
528,0,0,0,1,0,0,0,0,2,0,...,0,0,1,Adam Smith,D,H,0.111111,0.144444,0.000000,Washington 9th District


In [424]:
bigram_count.to_csv("bigram_count.csv")

In [427]:
bigrams.head()

,custom_id,twitter_handle,bigram
0,263,RepFinkenauer,"(health,care)"
1,263,RepFinkenauer,"(care,act)"
2,263,RepFinkenauer,"(million,americans)"
3,263,RepFinkenauer,"(pre,existing)"
4,263,RepFinkenauer,"(existing,conditions)"


In [454]:
journdf.tail()

,"(000,jobs)","(100,days)","(116th,congress)","(2019,congressional)","(21st,century)","(6,million)","(9,11)","(abortion,survivors)","(access,quality)","(act,2019)",...,"(watch,live)","(white,house)","(women,act)",name,party,position,score_0,score_115,score_116,state
custom_id,,,,,,,,,,,,,,,,,,,,,
2095,0,0,0,0,0,0,0,0,0,0,...,1,0,0,Steven Crowder,R,J,r_solid,None,None,FoxNewsOpinion
2096,0,0,0,0,0,0,0,0,0,0,...,0,1,0,J. Kenneth Blackwell,R,J,r_solid,None,None,FoxNewsOpinion
2098,0,0,0,0,0,0,0,0,0,0,...,0,0,0,CalThomas,R,J,r_solid,None,None,FoxNewsOpinion
2099,0,0,0,0,0,0,1,0,0,0,...,0,0,0,Glenn Beck,R,J,r_solid,None,None,FoxNewsOpinion
2100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Dan Gainor,R,J,r_solid,None,None,FoxNewsOpinion
